In [1]:
#import things

from deepdiff import DeepDiff
from Bio import SeqIO

#Parse both files using SeqIO for a high level comparison

for bopindex, boprecord in enumerate(SeqIO.parse("/Users/muyao/Downloads/stuff/new/bop27_1_4.gb", "genbank")):
    print("bopndex %i, ID = %s, length %i, with %i features"
          % (bopindex, boprecord.id, len(boprecord.seq), len(boprecord.features)))
for ncindex, ncrecord in enumerate(SeqIO.parse("/Users/muyao/Downloads/stuff/new/NC_000913_3.gb", "genbank")):
    print("ncindex %i, ID = %s, length %i, with %i features"
          % (ncindex, ncrecord.id, len(ncrecord.seq), len(ncrecord.features)))
    
#Only care about the features and their qualifiers

bopfeatures = boprecord.features
ncfeatures = ncrecord.features

# the gene_synonym entry in nc are in a ; separated format because it was all just listed under one line in the original gb file.
# we're splitting them up into a list to match the format of bop27

for feature in ncfeatures:
    if ('gene_synonym' in feature.qualifiers.keys() and ';' in feature.qualifiers['gene_synonym'][0]):
        synonym_list = feature.qualifiers['gene_synonym'][0]
        feature.qualifiers['gene_synonym'] = synonym_list.split("; ")

# Let's separate the features into a dictionary tree by the type of feature and set the locus_tag as a key of sorts for ones with locus tags

def split_features(features):
    feature_dic = {}
    for feature in features:
        if ('locus_tag' in feature.qualifiers.keys() ):
            if feature.type not in feature_dic:
                feature_dic[feature.type] = {}
            if isinstance(feature_dic[feature.type], list):
                feature_dic[feature.type].append(feature)
            elif feature.qualifiers['locus_tag'][0] not in feature_dic[feature.type].keys():
                feature_dic[feature.type][feature.qualifiers['locus_tag'][0]] = feature
            else:
                raise ValueError('Non-Unique Locus_tag')
            feature_dic[feature.type] = feature_dic[feature.type]
        #And just have a list for the ones that don't with everything of the same type, in order
        else:
            if feature.type not in feature_dic:
                feature_dic[feature.type] = []
            try:
                feature_dic[feature.type].append(feature)
            except AttributeError:
                feature_dic[feature.type] = list(feature_dic[feature.type].values())
                feature_dic[feature.type].append(feature)
                
    return feature_dic
nc_dic = split_features(ncfeatures)
bop_dic = split_features(bopfeatures)
    
    

bopndex 0, ID = NC_000913.3 SBRG v1.4, length 4641653, with 9461 features
ncindex 0, ID = NC_000913.3, length 4641652, with 9461 features


In [2]:
#We need to check if all the keys match in all the fields both references share. Here thee locus_tags are the keys

print(bop_dic.keys())
print(nc_dic.keys())
intersection = list(set(bop_dic.keys()) & set(nc_dic.keys()))
for each in intersection:
    
    if isinstance(bop_dic[each], dict):
        print("Dict: " + str(each) +": "+ str(bop_dic[each].keys() == nc_dic[each].keys()) +" "+ str(len(bop_dic[each].keys())) + ":" +str(len(nc_dic[each].keys())))
    else:
        print("List: "+str(each) +": "+ str(bop_dic[each] == nc_dic[each]) +" "+ str(len(bop_dic[each])) + ":" +str(len(nc_dic[each])))

#Now for the rest--things in nc but not bop:
intersection = list(set(bop_dic.keys()) & set(nc_dic.keys()))
symmetric_difference = (bop_dic.keys() ^ nc_dic.keys())

ncsd = []
bopsd = []

for each in symmetric_difference:
    print(each)

    if each in nc_dic.keys():
        print("nc: "+str(len(nc_dic[each])))
        #print(nc_dic[each])
        ncsd.append(nc_dic[each])
    else:
        print("bop: "+str(len(bop_dic[each])))
        #print(bop_dic[each])
        bopsd.append(bop_dic[each])

dict_keys(['gene', 'CDS', 'repeat_region', 'D_segment', 'ncRNA', 'STS', 'rRNA', 'tRNA', 'tmRNA', 'rep_origin'])
dict_keys(['source', 'gene', 'CDS', 'repeat_region', 'mobile_element', 'ncRNA', 'STS', 'rRNA', 'tRNA', 'misc_feature', 'tmRNA', 'rep_origin'])
Dict: tRNA: True 89:89
Dict: tmRNA: True 2:2
List: repeat_region: False 355:355
List: STS: False 49:49
Dict: ncRNA: True 65:65
List: rep_origin: False 1:1
Dict: CDS: True 4319:4319
Dict: gene: True 4498:4498
Dict: rRNA: True 22:22
source
nc: 1
mobile_element
nc: 49
D_segment
bop: 61
misc_feature
nc: 11


In [3]:
#now that we know the keys are the same, we can just go through each dictionary with the same keys
#Let's just start with comparing features of type 'gene' for now

locus_tags = bop_dic['gene'].keys()

#This is what the comparisons look like:
test_tag = list(locus_tags)[0]

bopgene = dict(bop_dic['gene'][test_tag].qualifiers)
ncgene = dict(nc_dic['gene'][test_tag].qualifiers)

print (bop_dic['gene'][test_tag].qualifiers)
print (nc_dic['gene'][test_tag].qualifiers)

print(bopgene == ncgene)

#Here we check for all objects

for feattype in intersection:
    #Compare the dictionaries(ones with locus_tags across to use as a key)
    if isinstance(bop_dic[feattype], dict):
        print("Now processing Dict: " + feattype)
        locus_tags = bop_dic[feattype].keys()
        for locus_tag in locus_tags:
            bopqual = dict(bop_dic[feattype][locus_tag].qualifiers)
            ncqual = dict(nc_dic[feattype][locus_tag].qualifiers)
            if( bopqual!=ncqual ):
                print("difference detected in ")
                print(locus_tag + feattype)
                print(bopqual['EC_number'] ," bop:nc ",ncqual['EC_number'])
                print(DeepDiff(bopqual,ncqual))

OrderedDict([('db_xref', ['EcoGene:EG11277', 'GeneID:944742']), ('gene', ['thrL']), ('gene_synonym', ['ECK0001', 'JW4367']), ('locus_tag', ['b0001'])])
OrderedDict([('gene', ['thrL']), ('locus_tag', ['b0001']), ('gene_synonym', ['ECK0001', 'JW4367']), ('db_xref', ['EcoGene:EG11277', 'GeneID:944742'])])
True
Now processing Dict: tRNA
Now processing Dict: tmRNA
Now processing Dict: ncRNA
Now processing Dict: CDS
difference detected in 
b0002CDS
['1.1.1.3']  bop:nc  ['1.1.1.3', '2.7.2.4']
{'iterable_item_added': {"root['EC_number'][1]": '2.7.2.4'}}
difference detected in 
b0025CDS
['2.7.1.26']  bop:nc  ['2.7.1.26', '2.7.7.2']
{'iterable_item_added': {"root['EC_number'][1]": '2.7.7.2'}}
difference detected in 
b0118CDS
['4.2.1.3']  bop:nc  ['4.2.1.3', '4.2.1.99']
{'iterable_item_added': {"root['EC_number'][1]": '4.2.1.99'}}
difference detected in 
b0149CDS
['2.4.1.129']  bop:nc  ['2.4.1.129', '3.4.-.-']
{'iterable_item_added': {"root['EC_number'][1]": '3.4.-.-'}}
difference detected in 
b0

# This is at this point no longer a set of compare the differences but a challenge of what i need to do to make the two the same.

#Seems like the same thing as the "gene_synoym" thing. But with 'EC_number'

But upon more careful inspection, bop27 seems to only have the first entry in the EC_number.

Instead of trying to make them the same/remove them. It seems to be the only difference in the qualifiers. Let's just note it and move on

# On to the lists!

In [4]:
#These don't have any keys to go off of. Some in the same category would have locus tages but not all. Thus we will have to just go in order and hope for the best. 
#It should be fine as the relative positions should stay the same

for feattype in intersection:
    #Compare the dictionaries(ones with locus_tags across to use as a key)
    if isinstance(bop_dic[feattype], list):
        print("Now processing List: " + feattype)
        length = len(bop_dic[feattype])
        for position in range(length):
            bopqual = dict(bop_dic[feattype][position].qualifiers)
            ncqual = dict(nc_dic[feattype][position].qualifiers)
            if( bopqual!=ncqual ):
                print("difference detected in ")
                print(locus_tag + feattype)
                print(bopqual ," bop:nc ",ncqual)
                print(DeepDiff(bopqual,ncqual))

Now processing List: repeat_region
Now processing List: STS
Now processing List: rep_origin


# Now for everything else:

There's a good chance the symmetric differences contain the same informations. Just in bop27 everything else is just pushed under the same type: D_segment
    

In [5]:
nceverythingelse = []
bopeverythingelse = []

for each in symmetric_difference:
    print(each)

    if each in nc_dic.keys():
        for feature in nc_dic[each]:
            nceverythingelse.append(dict(feature.qualifiers))

    else:
        for feature in bop_dic[each]:
            bopeverythingelse.append(dict(feature.qualifiers))
            
DeepDiff(nceverythingelse,bopeverythingelse)
ctr = 0

for each in (bopeverythingelse):
    if each in nceverythingelse:
        nceverythingelse.remove(each)
        
    ctr +=1
print(ctr, " analyzed ", len(nceverythingelse), "left")

source
mobile_element
D_segment
misc_feature
61  analyzed  0 left


# The qualifiers are the same with the following exceptions:

NC has a singular field with multiple values separated by ";" as opposed to multiple fields with the same name for each value in BOP. In the parsing there were inconsistencies to how it's handled.

D_segment vs source, misc_feature, mobile_element

